# Covid-19 Analysis

<h4>Setup and read dataset file. Souce in Kaggle, "imdevskp/corona-virus-report"
<h5>    
    Plotly is required. You'll need a username and corresponding api_key to connect to it and generate the plots online.

In [15]:
import chart_studio, zipfile, math
import numpy as np
import pandas as pd
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.io as pio

In [16]:
initable = pd.read_table('covid-19.ini', delimiter=',', dtype={'Key':str, 'Value':str})
chart_studio.tools.set_credentials_file(username=initable.iloc[0,1], api_key=initable.iloc[1,1])
z = zipfile.ZipFile(initable.iloc[2,1])
dataset = pd.read_csv(z.open(initable.iloc[3,1]))

In [17]:
del dataset["Lat"]
del dataset["Long"]
dataset.rename({"Province/State": "Province"}, axis=1, inplace=True)
dataset.rename({"Country/Region": "Country"}, axis=1, inplace=True)
interestingCountries=["France","Brazil", "US", "Italy", "Spain", "Japan", "Korea"]
series = []

<h4>drop non-interesting Countries

In [18]:
dataset=dataset[dataset.Country.isin(interestingCountries)]
dataset

,Province,Country,Date,Confirmed,Deaths,Recovered
28,NaN,Brazil,1/22/20,0,0,0
107,French Guiana,France,1/22/20,0,0,0
108,French Polynesia,France,1/22/20,0,0,0
109,Guadeloupe,France,1/22/20,0,0,0
110,Mayotte,France,1/22/20,0,0,0
...,...,...,...,...,...,...
19377,NaN,Italy,4/5/20,128948,15887,21815
19379,NaN,Japan,4/5/20,3139,77,514
19441,NaN,Spain,4/5/20,131646,12641,38080
19465,NaN,US,4/5/20,337072,9619,17448


<h5>Group states in a country

In [19]:
confirmedresultset=dataset.groupby(['Country','Date']).agg({'Confirmed': sum})
confirmedresultset.reset_index(level=['Date'], inplace=True) #put Date back to column
confirmedresultset['Date']=pd.to_datetime(confirmedresultset['Date'], format='%m/%d/%y') #convert column to type Date
confirmedresultset=confirmedresultset.sort_values(['Date'], ascending=True) #sort by Date
confirmedresultset.tail(len(interestingCountries))

,Date,Confirmed
Country,,
France,2020-04-04,90848
Italy,2020-04-05,128948
Japan,2020-04-05,3139
France,2020-04-05,93773
Brazil,2020-04-05,11130
Spain,2020-04-05,131646
US,2020-04-05,337072


In [20]:
deathsresultset=dataset.groupby(['Country','Date']).agg({'Deaths': sum})
deathsresultset.reset_index(level=['Date'], inplace=True) #put Date back to column
deathsresultset['Date']=pd.to_datetime(deathsresultset['Date'], format='%m/%d/%y') #convert column to type Date
deathsresultset=deathsresultset.sort_values(['Date'], ascending=True) #sort by Date
deathsresultset.tail(len(interestingCountries))

,Date,Deaths
Country,,
France,2020-04-04,7574
Italy,2020-04-05,15887
Japan,2020-04-05,77
France,2020-04-05,8093
Brazil,2020-04-05,486
Spain,2020-04-05,12641
US,2020-04-05,9619


In [21]:
recoveredresultset=dataset.groupby(['Country','Date']).agg({'Recovered': sum})
recoveredresultset.reset_index(level=['Date'], inplace=True) #put Date back to column
recoveredresultset['Date']=pd.to_datetime(recoveredresultset['Date'], format='%m/%d/%y') #convert column to type Date
recoveredresultset=recoveredresultset.sort_values(['Date'], ascending=True) #sort by Date
recoveredresultset.tail(len(interestingCountries))

,Date,Recovered
Country,,
France,2020-04-04,15572
Italy,2020-04-05,21815
Japan,2020-04-05,514
France,2020-04-05,16349
Brazil,2020-04-05,127
Spain,2020-04-05,38080
US,2020-04-05,17448


<h4>Data for confirmed cases graphic

In [22]:
series = []
for country in interestingCountries: 
    series.append(
        go.Scatter(
            x=confirmedresultset[confirmedresultset.index == country].Date, 
            y=confirmedresultset[confirmedresultset.index == country].Confirmed, 
            name=country+" Confirmed", 
            mode="markers+lines",
            marker={"symbol":"circle", "size": 5},
            )
        )

In [23]:
fig = go.Figure(series)
fig.update_layout(title_text='COVID-19 Confirmed cases by country')
fig.update_xaxes(title_text='Days')
fig.update_yaxes(title_text='Confirmed cases', type='log')
fig.add_annotation(
        x='2020-03-24',
        y=math.log(2247,10), xref="x", yref="y",
        text="SP lockdown", textangle = 330, showarrow=True, arrowcolor='grey')
fig.add_annotation(
        x='2020-03-09',
        y=math.log(9172,10), xref="x", yref="y",
        text="Italy lockdown", textangle = 330, showarrow=True, arrowcolor='grey')
fig.add_annotation(
        x='2020-03-11',
        y=math.log(2293,10), xref="x", yref="y",
        text="France lockdown", textangle = 330, showarrow=True, arrowcolor='grey')
fig.add_annotation(
        x='2020-03-19',
        y=math.log(13677,10), xref="x", yref="y",
        text="Calif. lockdown", textangle = 330, showarrow=True, arrowcolor='grey')
py.iplot(fig, file='covid-19.html', auto_open=False)

Data for Death cases

In [24]:
series = []
for country in interestingCountries: 
    series.append(
        go.Scatter(
            x=deathsresultset[deathsresultset.index == country].Date, 
            y=deathsresultset[deathsresultset.index == country].Deaths, 
            name=country+" Deaths",
            mode="markers+lines",
            marker={"symbol":"circle", "size": 5},
        )
    )

In [25]:
fig = go.Figure(series)
fig.update_layout(title_text='COVID-19 Deaths by country')
fig.update_xaxes(title_text='Days')
fig.update_yaxes(title_text='Deaths', type='log')
py.iplot(fig, file='covid-19.html', auto_open=False)

<h4>Recovered cases

In [26]:
series = []
for country in interestingCountries: 
    series.append(
        go.Scatter(
            x=recoveredresultset[recoveredresultset.index == country].Date, 
            y=recoveredresultset[recoveredresultset.index == country].Recovered, 
            name=country+" Recovered",
            mode="markers+lines",
            marker={"symbol":"circle", "size": 5},
        )
    )

In [27]:
fig = go.Figure(series)
fig.update_layout(title_text='COVID-19 Recovered by country')
fig.update_xaxes(title_text='Days')
fig.update_yaxes(title_text='Recovered', type='log')
py.iplot(fig, file='covid-19.html', auto_open=False)